<h1>Unloading preprocessed data from the data mart</h1>

In [2]:
import numpy as np
import pandas as pd
import os

from dotenv import load_dotenv
from clickhouse_driver import Client

In [3]:
load_dotenv()

HOST = os.getenv('ANALYTICAL_HOST')
USERNAME = os.getenv('USERNAME')
PASSWORD = os.getenv('ANALYTICAL_PASSWORD')
CA = os.getenv('CA')
READONLY = os.getenv('READONLY')

client = Client(host=HOST, user=USERNAME, password=PASSWORD, ca_certs=CA, secure=True)

In [4]:
df = client.query_dataframe(f'''
    SELECT time, temp_ice, temp_inside, temp_outside, hum_inside, hum_outside
    FROM akarmanov_test_db.genesis_arena_prediction_ice
''')

 
# df[10000:50000].to_csv('../data/small_data.csv', index=False)
# df = pd.read_csv('../data/small_data.csv')

In [122]:
df = df.drop_duplicates(subset='time').sort_values(by='time', ignore_index=True).dropna()
df.to_csv('../data/dataset_2022-09-20_2024-08-03.csv', index=False)
df[10000:50000].to_csv('../data/small_data.csv', index=False)

In [37]:
start_time = df['time'].min().strftime('%Y-%m-%d %H:%M:00')
end_time = df['time'].max().strftime('%Y-%m-%d %H:%M:00')

print(start_time, end_time, sep='\n')

2022-09-20 11:31:00
2024-08-03 23:10:00


In [54]:
genTimeSeries = pd.Series(pd.date_range(start_time, end_time, freq='1min'))
dfTimeSeries = df['time']

genTimeSeries[~genTimeSeries.isin(dfTimeSeries)]

43890    2022-10-20 23:01:00
43891    2022-10-20 23:02:00
43892    2022-10-20 23:03:00
43893    2022-10-20 23:04:00
43894    2022-10-20 23:05:00
                 ...        
786442   2024-03-19 14:53:00
786443   2024-03-19 14:54:00
786444   2024-03-19 14:55:00
786445   2024-03-19 14:56:00
786446   2024-03-19 14:57:00
Length: 534, dtype: datetime64[ns]

In [75]:
genTimeSeries[~genTimeSeries.isin(dfTimeSeries)].apply(lambda x: x.date()).value_counts()

2024-03-19    180
2022-10-20     59
2022-10-30     59
2022-12-20     29
2023-08-13     29
2023-09-19     29
2023-08-05     29
2023-04-30     29
2024-02-27     29
2024-02-13     29
2023-04-01     28
2022-11-09      1
2023-05-01      1
2022-12-21      1
2023-08-14      1
2024-02-28      1
Name: count, dtype: int64

# Add missing data from main database to data mart

In [58]:
import numpy as np
import pandas as pd
import os
import datetime as dt
from datetime import datetime
from dotenv import load_dotenv
from clickhouse_driver import Client
from data_wrapper import db_wrapper

load_dotenv()

HOST = os.getenv('BASE_HOST')
USERNAME = os.getenv('USERNAME')
PASSWORD = os.getenv('BASE_PASSWORD')
CA = os.getenv('CA')
READONLY = os.getenv('READONLY')
ANALYTICAL_HOST = os.getenv('ANALYTICAL_HOST')
ANALYTICAL_PASSWORD = os.getenv('ANALYTICAL_PASSWORD')

# Подключаемся к боевой базе данных
wrap = db_wrapper.ClickHouseWrapper(host=HOST, user=USERNAME, password=PASSWORD, ca=CA)
# Подключаемся к аналитической БД с правами загрузки
client = Client(host=ANALYTICAL_HOST, user=USERNAME, password=ANALYTICAL_PASSWORD, ca_certs=CA, secure=True)

In [102]:
sensors = [
# Потребление ввода холодильной машины
{
    "db": "genesis_arena", "name": "map12e_142", "measurement": "Total AP energy", 
    "channel": 1, "phase": 0, "name_in_df": "cm_consumption", "mode": "max-min"
},
# Мгновенная мощность компрессоров
{
    "db": "genesis_arena", "name": "map12e_142", "measurement": "Total P", 
    "channel": 2, "phase": 0, "name_in_df": "state1", "mode": "max-min"
},
{
    "db": "genesis_arena", "name": "map12e_142", "measurement": "Total P",
    "channel": 3, "phase": 0, "name_in_df": "state2", "mode": "max-min"
},
{
    "db": "genesis_arena", "name": "map12e_142", "measurement": "Total P",
    "channel": 4, "phase": 0, "name_in_df": "state3", "mode": "max-min"
},
{
    "db": "genesis_arena", "name": "map12e_145", "measurement": "Total P",
    "channel": 1, "phase": 0, "name_in_df": "state4", "mode": "max-min"
},
# Мгновенная мощность конденсаторов
{
    "db": "genesis_arena", "name": "map12e_23", "measurement": "Total P", 
    "channel": 1, "phase": 0, "name_in_df": "condensator4", "mode": "mean"
},
{
    "db": "genesis_arena", "name": "map12e_49", "measurement": "Total P", 
    "channel": 2, "phase": 0, "name_in_df": "condensator1", "mode": "mean"
},
{
    "db": "genesis_arena", "name": "map12e_49", "measurement": "Total P", 
    "channel": 3, "phase": 0, "name_in_df": "condensator2", "mode": "mean"
},
{
    "db": "genesis_arena", "name": "map12e_49", "measurement": "Total P", 
    "channel": 4, "phase": 0, "name_in_df": "condensator3", "mode": "mean"
},
# Мощность циркуляционного насоса
{
    "db": "genesis_arena", "name": "map12e_145", "measurement": "Total P", 
    "channel": 3, "phase": 0, "name_in_df": "power_pump", "mode": "mean"
},
# Температура поверхности льда
{
    "db": "genesis_arena", "name": "msw-v3_2", "measurement": "Temperature", 
    "channel": 0, "phase": 0, "name_in_df": "temp_ice", "mode": "mean"
},
# Внешняя температура
{
    "db": "genesis_arena", "name": "weather_owm", "measurement": "Temperature",
    "channel": 0, "phase": 0, "name_in_df": "temp_outside", "mode": "mean"
},
# Внешняя влажность
{
    "db": "genesis_arena", "name": "weather_owm", "measurement": "Humidity",
    "channel": 0, "phase": 0, "name_in_df": "hum_outside", "mode": "mean"
},
# Внутреняя температура 
{
    "db": "genesis_arena", "name": "msw-v3_175", "measurement": "Temperature",
    "channel": 0, "phase": 0, "name_in_df": "temp_inside", "mode": "mean"
},
# Внутреняя влажность 
{
    "db": "genesis_arena", "name": "msw-v3_175", "measurement": "Humidity",
    "channel": 0, "phase": 0, "name_in_df": "hum_inside", "mode": "mean"
}
]

In [ ]:
presample = dt.timedelta(minutes=1)
missing_days = genTimeSeries[~genTimeSeries.isin(dfTimeSeries)].apply(lambda x: x.date()).unique()

df = pd.DataFrame()
for missing_day in missing_days:
    start = dt.datetime.combine(missing_day, dt.time(0, 0, 0)) - dt.timedelta(hours=3)
    end = start + dt.timedelta(days=1)

    dfi = wrap.get_particular_sensors(start, end, sensors, presample_time=presample,  without_confidence=True)
    df = pd.concat([df, dfi], axis=0)

In [104]:
df.head()

,cm_consumption,state1,state2,state3,state4,condensator4,condensator1,condensator2,condensator3,power_pump,temp_ice,temp_outside,hum_outside,temp_inside,hum_inside
2022-10-20 00:00:00+03:00,1.232292,36563.352539,36045.404297,37111.740234,0.470,NaN,NaN,NaN,NaN,14488.245117,-1.7000,5.809,85.979167,NaN,NaN
2022-10-20 00:01:00+03:00,2.281250,36844.675781,35957.416016,37708.455078,0.395,NaN,NaN,NaN,NaN,14500.530273,-1.7075,5.805,85.895833,NaN,57.124999
2022-10-20 00:02:00+03:00,2.274479,36968.638672,36211.759766,37841.830078,0.335,NaN,NaN,NaN,NaN,14540.429688,-1.7175,5.801,85.812500,NaN,57.175000
2022-10-20 00:03:00+03:00,2.280208,36390.483887,36110.509766,37715.000000,0.350,NaN,NaN,NaN,NaN,14536.700195,-1.7725,5.797,85.729167,NaN,57.150000
2022-10-20 00:04:00+03:00,2.280729,36855.826172,35705.308594,37094.234375,0.445,NaN,NaN,NaN,NaN,14485.524902,-1.7825,5.793,85.645833,NaN,57.162500


In [105]:
df = (df
 .tz_localize(None)
 .reset_index(names=['time'])
 .assign(temp_ice=lambda df: np.where(df['temp_ice'].between(-6, 3), df['temp_ice'], np.NaN))
 .pipe(lambda df: df.ffill().bfill())
 .assign(power_compressors=lambda df: df[['state1', 'state2', 'state3', 'state4']].sum(axis=1))
 .assign(power_condensators=lambda df: df[['condensator1', 'condensator2', 'condensator3', 'condensator4']].sum(axis=1))
 .assign(prediction_temp_ice=lambda df: np.NaN)
)

client.execute('USE akarmanov_test_db')
client.insert_dataframe('''
INSERT INTO "genesis_arena_prediction_ice" (
    time, temp_ice, power_compressors, power_condensators, power_pump,
    cm_consumption, temp_outside, temp_inside, hum_outside, hum_inside,
    prediction_temp_ice
) VALUES
''',
    df[['time', 'temp_ice', 'power_compressors', 'power_condensators', 'power_pump',
    'cm_consumption', 'temp_outside', 'temp_inside', 'hum_outside', 'hum_inside',
    'prediction_temp_ice']],
    settings=dict(use_numpy=True)
)

22686

In [106]:
df = client.query_dataframe(f'''
    SELECT time, temp_ice, temp_inside, temp_outside, hum_inside, hum_outside
    FROM akarmanov_test_db.genesis_arena_prediction_ice
''')

 
df = df.drop_duplicates(subset='time').sort_values(by='time', ignore_index=True).dropna()

In [117]:
genTimeSeries = pd.Series(pd.date_range(start_time, end_time, freq='1min'))
df = df.merge(pd.DataFrame({'time': genTimeSeries}), how='right').ffill()

In [128]:
df.to_csv('../data/dataset_2022-09-20_2024-08-03.csv', index=False)
df[10000:50000].to_csv('../data/small_data.csv', index=False)